# Notebook 3: Funções de Transformação e Limpeza de Dados

## Introdução
Neste notebook, iremos implementar as transformações e limpar os dados da tabela 'ReportSource' do conjunto de dados FAERS, com base na análise exploratória realizada no notebook anterior (Exploratory_Data_Analysis_ReportSource). 

## Estrutura do notebook
- Importação de Bibliotecas
- Carregamento dos Dados (Parquet Raw) usando o Schema definido 
- Aplicar apenas as Transformações necessárias
- Salvar **Parquet Final** com os Dados Limpos


In [0]:
# Importar bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, LongType
from pyspark.sql.functions import col
from pyspark.sql.functions import trim 

In [0]:
%run "../Funcoes_auxiliares" 

In [0]:
# Carregar dados do Parquet raw
schema_rpsr = StructType([
    StructField("primaryid", LongType(), False,{'description': "Unique number for identifying a FAERS report."}),
    StructField("caseid", IntegerType(), False,{'description': "Number for identifying a FAERS case."}),
    StructField("rpsr_cod", StringType(), False, {'description': "Code for the source of the report"})
    ])
df_rpsr = spark.read.format('parquet').schema(schema_rpsr).load('dbfs:/FileStore/FAERS-grupo-4/rpsr_raw')

In [0]:
#'Descascar' os espaços presentes nos registos de código de report
df_rpsr=df_rpsr.withColumn('rpsr_cod',trim(df_rpsr['rpsr_cod']))

In [0]:
# Criar DataFrame de mapeamento de códigos para descrição das fontes
mapping_data = [
    ("FGN", "Foreign"),
    ("SDY", "Study"),
    ("LIT", "Literature"),
    ("CSM", "Consumer"),
    ("HP", "Health Professional"),
    ("UF", "User Facility"),
    ("CR", "Company Representative"),
    ("DT", "Distributor"),
    ("OTH", "Other")
]
mapping_columns = ["rpsr_cod", "report_source"]
df_mapping = spark.createDataFrame(mapping_data, mapping_columns)

# Join entre df_rpsr e o DataFrame de mapeamento
df_rpsr = df_rpsr.join(df_mapping, on="rpsr_cod", how="left")

# Organizar o DataFrame resultante
df_rpsr=df_rpsr['primaryid', 'caseid','rpsr_cod', 'report_source']


In [0]:
df_rpsr.display()

primaryid,caseid,rpsr_cod,report_source
214020341,21402034,HP,Health Professional
214020451,21402045,CSM,Consumer
214021021,21402102,CSM,Consumer
214021051,21402105,CSM,Consumer
214021571,21402157,CSM,Consumer
214022161,21402216,CSM,Consumer
214025461,21402546,HP,Health Professional
214032641,21403264,CSM,Consumer
214033011,21403301,FGN,Foreign
214033011,21403301,CSM,Consumer


In [0]:
#Restaurar metadados
df_rpsr = restore_metadata(schema_rpsr,df_rpsr)

In [0]:
# salvar parquet final
df_rpsr.write.mode('overwrite').parquet("dbfs:/FileStore/FAERS-grupo-4/rpsr_final")